In [7]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [8]:
import logging
import time

In [9]:
logging.basicConfig(
    filename='logs/ingestion_db.log',
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='a'
)

In [12]:
engine = create_engine('sqlite:///inventory.db')

In [11]:
from google.colab import drive


In [13]:
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
def ingest_db(df, table_name,engine):
    df.to_sql(table_name, engine, if_exists='replace', index=False)

In [25]:
import pandas as pd
csv_files = [
    '/content/drive/MyDrive/Inventory_data/begin_inventory.csv',
    '/content/drive/MyDrive/Inventory_data/end_inventory.csv',
    '/content/drive/MyDrive/Inventory_data/vendor_invoice.csv',
    '/content/drive/MyDrive/Inventory_data/purchase_prices.csv',
]
for file in csv_files:
    df = pd.read_csv(file)
    table_name = file.split('/')[-1].replace('.csv', '')
    print(f"shape: {df.shape}")
    ingest_db(df, table_name, engine)

shape: (206529, 9)
shape: (224489, 9)
shape: (5543, 10)
shape: (12261, 9)


In [23]:
import pandas as pd
csv_files = [
    '/content/drive/MyDrive/Inventory_data/purchases.csv',
]
for file in csv_files:
    df = pd.read_csv(file)
    table_name = file.split('/')[-1].replace('.csv', '')
    print(f"shape: {df.shape}")
    ingest_db(df, table_name, engine)

shape: (2372474, 16)


In [18]:
import pandas as pd
import gc

def optimize_df(df):
    for col in df.select_dtypes(include=['int']).columns:
        df[col] = pd.to_numeric(df[col], downcast='unsigned')
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].nunique() / len(df[col]) < 0.5:
            df[col] = df[col].astype('category')
    return df

csv_files = [
    '/content/drive/MyDrive/Inventory_data/sales.csv',
]

chunk_size = 5000000  # Adjust based on your memory capacity

for file in csv_files:
    table_name = file.split('/')[-1].replace('.csv', '')
    try:
        for chunk in pd.read_csv(file, chunksize=chunk_size):
            chunk = optimize_df(chunk)
            ingest_db(chunk, table_name, engine)
            del chunk
            gc.collect()
    except Exception as e:
        print(f"Error processing {file}: {e}")


In [19]:
import pandas as pd
import sqlite3

In [20]:
#creating database connection
conn = sqlite3.connect('inventory.db')

In [33]:
# Checking tables present in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(tables)

                                                 name
0   /content/drive/MyDrive/Inventory_data/begin_in...
1   /content/drive/MyDrive/Inventory_data/end_inve...
2   /content/drive/MyDrive/Inventory_data/vendor_i...
3   /content/drive/MyDrive/Inventory_data/purchase...
4     /content/drive/MyDrive/Inventory_data/purchases
5                                               sales
6                                           purchases
7                                     begin_inventory
8                                       end_inventory
9                                      vendor_invoice
10                                    purchase_prices


In [34]:
pd.read_sql("select count(*) from sales", conn)

,count(*)
0,2825363


In [37]:
# Use the correct table names from the 'tables' DataFrame
valid_tables = tables['name'].tolist()[-6:] # Assuming the last 5 entries are the correct table names

for table in valid_tables:
  print('-'*50, f'{table}','-'*50)
  print('Count of records:', pd.read_sql(f"select count(*) as count from {table}", conn)['count'].values[0])
  display(pd.read_sql(f"select * from {table} limit 5", conn))

-------------------------------------------------- sales --------------------------------------------------
Count of records: 2825363


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,63_SWORDBREAK_8329,63,8329,R Mondavi Chard Pvt Sel,750mL,1,6.98,6.98,2024-10-07,750.0,2,0.11,1392,CONSTELLATION BRANDS INC
1,63_SWORDBREAK_8329,63,8329,R Mondavi Chard Pvt Sel,750mL,1,6.98,6.98,2024-10-10,750.0,2,0.11,1392,CONSTELLATION BRANDS INC
2,63_SWORDBREAK_8329,63,8329,R Mondavi Chard Pvt Sel,750mL,1,6.98,6.98,2024-10-13,750.0,2,0.11,1392,CONSTELLATION BRANDS INC
3,63_SWORDBREAK_8329,63,8329,R Mondavi Chard Pvt Sel,750mL,4,27.92,6.98,2024-10-18,750.0,2,0.45,1392,CONSTELLATION BRANDS INC
4,63_SWORDBREAK_8329,63,8329,R Mondavi Chard Pvt Sel,750mL,2,13.96,6.98,2024-10-20,750.0,2,0.22,1392,CONSTELLATION BRANDS INC


-------------------------------------------------- purchases --------------------------------------------------
Count of records: 2372474


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1


-------------------------------------------------- begin_inventory --------------------------------------------------
Count of records: 206529


...

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


-------------------------------------------------- vendor_invoice --------------------------------------------------
Count of records: 5543


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


-------------------------------------------------- purchase_prices --------------------------------------------------
Count of records: 12261


,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


In [38]:
purchases = pd.read_sql_query("select * from purchases where VendorNumber = 4466", conn)
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [42]:
purchase_prices = pd.read_sql_query("""select * from purchase_prices where VendorNumber = 4466""", conn)
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [43]:
vendor_invoice = pd.read_sql_query("""select * from vendor_invoice where VendorNumber = 4466""", conn)
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


In [46]:
sales = pd.read_sql_query("""select * from sales where VendorNo = 4466""", conn)
sales

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,64_CESTERFIELD_3140,64,3140,TGI Fridays Orange Dream,1.75L,1,12.99,12.99,2024-10-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
1,64_CESTERFIELD_3140,64,3140,TGI Fridays Orange Dream,1.75L,1,12.99,12.99,2024-10-31,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2,64_CESTERFIELD_5215,64,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-10-07,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
3,64_CESTERFIELD_5255,64,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-10-29,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
4,65_LUTON_3140,65,3140,TGI Fridays Orange Dream,1.75L,1,12.99,12.99,2024-10-01,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,9_BLACKPOOL_5215,9,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-12-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2011,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-02,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2012,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2013,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE


In [49]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

,,Quantity,Dollars
Brand,PurchasePrice,,
3140,11.19,4640,51921.60
5215,9.41,4923,46325.43
5255,9.35,6215,58110.25


In [50]:
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [52]:
vendor_invoice['PONumber'].nunique()

55

In [54]:
vendor_invoice.columns

Index(['VendorNumber', 'VendorName', 'InvoiceDate', 'PONumber', 'PODate',
       'PayDate', 'Quantity', 'Dollars', 'Freight', 'Approval'],
      dtype='object')

In [55]:
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [57]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()

,SalesDollars,SalesPrice,SalesQuantity
Brand,,,
3140,8612.37,6650.879883,663
5215,11535.12,8482.469851,888
5255,17276.70,11041.499805,1330


In [58]:
vendor_invoice.columns

Index(['VendorNumber', 'VendorName', 'InvoiceDate', 'PONumber', 'PODate',
       'PayDate', 'Quantity', 'Dollars', 'Freight', 'Approval'],
      dtype='object')

In [59]:
freight_summary = pd.read_sql_query("""select VendorNumber, SUM(Freight) as FreightCost
from vendor_invoice
Group By VendorNumber""", conn)
freight_summary

,VendorNumber,FreightCost
0,2,27.08
1,54,0.48
2,60,367.52
3,105,62.39
4,200,6.19
...,...,...
121,98450,856.02
122,99166,130.09
123,172662,178.34
124,173357,202.50


In [62]:
pd.read_sql_query("""select
  p.VendorNumber,
  p.VendorName,
  p.Brand,
  p.PurchasePrice,
  pp.Volume,
  pp.Price as ActualPrice,
  SUM(p.Quantity) as TotalPurchaseQuantity,
  SUM(p.Dollars) as TotalPurchaseDollars
  FROM purchases p
  JOIN purchase_prices pp
  ON p.Brand = pp.Brand
  WHERE p.PurchasePrice >0
  GROUP BY p.VendorNumber, p.VendorName, p.Brand
  ORDER BY TotalPurchaseDollars""", conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars
0,7245,PROXIMO SPIRITS INC.,3065,0.71,50,0.99,1,0.71
1,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,1,1.47
2,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,2,1.48
3,8004,SAZERAC CO INC,5683,0.39,50,0.49,6,2.34
4,9815,WINE GROUP INC,8527,1.32,750,4.99,2,2.64
...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,138109,3023206.01
10688,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,201682,3261197.94
10689,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,187407,3418303.68
10690,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,164038,3804041.22


In [63]:
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

In [67]:
pd.read_sql_query("""SELECT
VendorNo,
Brand,
SUM(SalesDollars) as TotalSalesDollars,
SUM(SalesPrice) as TotalSalesPrice,
SUM(SalesQuantity) as TotalSalesQuantity,
SUM(ExciseTax) as TotalExciseTax
FROM sales
GROUP BY VendorNo, Brand
ORDER BY TotalSalesDollars""", conn)

,VendorNo,Brand,TotalSalesDollars,TotalSalesPrice,TotalSalesQuantity,TotalExciseTax
0,8004,3265,1.96,1.470000,4,0.200000
1,3924,9123,1.98,0.990000,2,0.100000
2,3960,8467,2.97,2.970000,3,0.150000
3,3960,8495,2.97,2.970000,3,0.150000
4,9206,3730,3.96,3.960000,4,0.200000
...,...,...,...,...,...,...
9372,17035,2589,983014.10,116765.465126,25790,47386.789917
9373,3960,3545,1013228.79,102078.279297,33521,61592.039796
9374,4425,3405,1051222.26,101761.479996,35774,65731.699493
9375,1128,1233,1220498.87,123025.225838,34613,63598.359702


In [72]:
final_table = pd.read_sql_query("""SELECT
      pp.VendorNumber,
      pp.Brand,
      pp.Price as "Actual Price",
      pp.PurchasePrice,
      SUM(s.SalesQuantity) AS TotalSalesQuantity,
      SUM(s.SalesDollars) AS TotalSalesDollars,
      SUM(s.SalesPrice) AS TotalSalesPrice,
      SUM(s.ExciseTax) AS TotalExciseTax,
      SUM(vi.Quantity) AS TotalVendorQuantity,
      SUM(vi.Dollars) AS TotalVendorDollars,
      SUM(vi.Freight) AS TotalFreight
      FROM purchase_prices pp
      JOIN sales s ON pp.VendorNumber = s.VendorNo AND pp.Brand = s.Brand
      JOIN vendor_invoice vi ON pp.VendorNumber = vi.VendorNumber AND pp.Brand = vi.Brand
      GROUP BY pp.VendorNumber, pp.Brand, pp.Price, pp.PurchasePrice""",conn)

DatabaseError: Execution failed on sql 'SELECT
      pp.VendorNumber,
      pp.Brand,
      pp.Price as "Actual Price",
      pp.PurchasePrice,
      SUM(s.SalesQuantity) AS TotalSalesQuantity,
      SUM(s.SalesDollars) AS TotalSalesDollars,
      SUM(s.SalesPrice) AS TotalSalesPrice,
      SUM(s.ExciseTax) AS TotalExciseTax,
      SUM(vi.Quantity) AS TotalVendorQuantity,
      SUM(vi.Dollars) AS TotalVendorDollars,
      SUM(vi.Freight) AS TotalFreight
      FROM purchase_prices pp
      JOIN sales s ON pp.VendorNumber = s.VendorNo AND pp.Brand = s.Brand
      JOIN vendor_invoice vi ON pp.VendorNumber = vi.VendorNumber AND pp.Brand = vi.Brand
      GROUP BY pp.VendorNumber, pp.Brand, pp.Price, pp.PurchasePrice': no such column: vi.Brand

In [74]:
vendor_sales_summary = pd.read_sql_query("""WITH FreightSummary AS(
  SELECT VendorNumber, SUM(Freight) AS FreightCost
  FROM vendor_invoice
  GROUP BY VendorNumber
),
PurchaseSummary AS(
  SELECT
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.Description,
    p.PurchasePrice,
    pp.Price as "ActualPrice",
    pp.Volume,
    SUM(p.Quantity) AS TotalPurchaseQuantity,
    SUM(p.Dollars) AS TotalPurchaseDollars
  FROM purchases p
  JOIN purchase_prices pp ON p.Brand = pp.Brand
  WHERE p.PurchasePrice > 0
  GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
  ),
SalesSummary AS(
  SELECT
    VendorNo,
    Brand,
    SUM(SalesQuantity) AS TotalSalesQuantity,
    SUM(SalesDollars) AS TotalSalesDollars,
    SUM(SalesPrice) AS TotalSalesPrice,
    SUM(ExciseTax) AS TotalExciseTax
  FROM sales
  GROUP BY VendorNo, Brand
)

SELECT
  ps.VendorNumber,
  ps.VendorName,
  ps.Brand,
  ps.Description,
  ps.PurchasePrice,
  ps."ActualPrice",
  ps.Volume,
  ps.TotalPurchaseQuantity,
  ps.TotalPurchaseDollars,
  ss.TotalSalesQuantity,
  ss.TotalSalesDollars,
  ss.TotalSalesPrice,
  ss.TotalExciseTax,
  fs.FreightCost
  FROM PurchaseSummary ps
  LEFT JOIN SalesSummary ss ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
  LEFT JOIN FreightSummary fs ON ps.VendorNumber = fs.VendorNumber
  ORDER BY ps.TotalPurchaseDollars DESC
""", conn)

In [91]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080,3811251.60,34613.0,1220498.87,123025.225838,63598.359702,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038,3804041.22,35774.0,1051222.26,101761.479996,65731.699493,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407,3418303.68,37773.0,894731.77,82569.639202,69404.909528,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682,3261197.94,60211.0,1275752.89,76734.849173,110634.689261,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109,3023206.01,33521.0,1013228.79,102078.279297,61592.039796,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750.0,2,2.64,0.0,0.00,0.000000,0.000000,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50.0,6,2.34,0.0,0.00,0.000000,0.000000,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50.0,2,1.48,2.0,1.98,0.990000,0.100000,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200.0,1,1.47,15.0,29.85,21.890000,3.150000,257032.07


In [92]:
vendor_sales_summary.to_csv('vendor_sales_summary.csv', index=False)

In [90]:
vendor_sales_summary.dtypes

,0
VendorNumber,int64
VendorName,object
Brand,int64
Description,object
PurchasePrice,float64
ActualPrice,float64
Volume,float64
TotalPurchaseQuantity,int64
TotalPurchaseDollars,float64
TotalSalesQuantity,float64


In [89]:
vendor_sales_summary.isnull().sum()

,0
VendorNumber,0
VendorName,0
Brand,0
Description,0
PurchasePrice,0
ActualPrice,0
Volume,0
TotalPurchaseQuantity,0
TotalPurchaseDollars,0
TotalSalesQuantity,0


In [88]:
vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP', 'MARTIGNETTI COMPANIES', 'PERNOD RICARD USA',
       'DIAGEO NORTH AMERICA INC', 'BACARDI USA INC',
       'JIM BEAM BRANDS COMPANY', 'MAJESTIC FINE WINES',
       'ULTRA BEVERAGE COMPANY LLP', 'STOLI GROUP,(USA) LLC',
       'PROXIMO SPIRITS INC.', 'MOET HENNESSY USA INC', 'CAMPARI AMERICA',
       'SAZERAC CO INC', 'CONSTELLATION BRANDS INC', 'M S WALKER INC',
       'SAZERAC NORTH AMERICA INC.', 'PALM BAY INTERNATIONAL INC',
       'REMY COINTREAU USA INC', 'SIDNEY FRANK IMPORTING CO',
       'E & J GALLO WINERY', 'WILLIAM GRANT & SONS INC',
       'HEAVEN HILL DISTILLERIES', 'DISARONNO INTERNATIONAL LLC',
       'EDRINGTON AMERICAS', 'CASTLE BRANDS CORP.',
       'SOUTHERN WINE & SPIRITS NE', 'STE MICHELLE WINE ESTATES',
       'TRINCHERO FAMILY ESTATES', 'MHW LTD', 'WINE GROUP INC',
       'PERFECTA WINES', 'LUXCO INC', 'TREASURY WINE ESTATES',
       'DIAGEO CHATEAU ESTATE WINES', 'SHAW ROSS INT L IMP LTD',
       'PINE STATE TRADING CO', 'PHILLIPS PRO

In [83]:
vendor_sales_summary['Description'].unique()

array(['Jack Daniels No 7 Black', "Tito's Handmade Vodka",
       'Absolut 80 Proof', ..., 'Crown Royal Apple',
       'Concannon Glen Ellen Wh Zin', 'The Club Strawbry Margarita'],
      dtype=object)

In [84]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype('float64')

In [85]:
vendor_sales_summary.fillna(0, inplace=True)

In [87]:
vendor_sales_summary['VendorName']=vendor_sales_summary['VendorName'].str.strip()

In [93]:
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [96]:
vendor_sales_summary['ProfitMargin'] = vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalSalesDollars']*100

In [97]:
vendor_sales_summary['StockTurnover'] = vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']

In [98]:
vendor_sales_summary['SalesToPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']

In [100]:
vendor_sales_summary.columns

Index(['VendorNumber', 'VendorName', 'Brand', 'Description', 'PurchasePrice',
       'ActualPrice', 'Volume', 'TotalPurchaseQuantity',
       'TotalPurchaseDollars', 'TotalSalesQuantity', 'TotalSalesDollars',
       'TotalSalesPrice', 'TotalExciseTax', 'FreightCost', 'GrossProfit',
       'ProfitMargin', 'StockTurnover', 'SalesToPurchaseRatio'],
      dtype='object')

In [99]:
cursor = conn.cursor()

In [101]:
cursor.execute("""CREATE TABLE vendor_sales_summary(
  VendorNumber INT,
  VendorName VARCHAR(100),
  Brand INT,
  Description VARCHAR(100),
  PurchasePrice FLOAT,
  "ActualPrice" FLOAT,
  Volume FLOAT,
  TotalPurchaseQuantity FLOAT,
  TotalPurchaseDollars FLOAT,
  TotalSalesQuantity FLOAT,
  TotalSalesDollars FLOAT,
  TotalSalesPrice FLOAT,
  TotalExciseTax FLOAT,
  FreightCost FLOAT,
  GrossProfit FLOAT,
  ProfitMargin FLOAT,
  StockTurnover FLOAT,
  SalesToPurchaseRatio FLOAT,
  PRIMARY KEY (VendorNumber, Brand)
)""")


In [102]:
pd.read_sql_query("select * from vendor_sales_summary", conn)

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnover,SalesToPurchaseRatio


In [103]:
vendor_sales_summary.to_sql('vendor_sales_summary', conn, if_exists='replace', index=False)

10692

In [104]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnover,SalesToPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080,3811251.60,34613.0,1220498.87,123025.225838,63598.359702,68601.68,-2590752.73,-212.269982,0.238579,0.320236
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038,3804041.22,35774.0,1051222.26,101761.479996,65731.699493,144929.24,-2752818.96,-261.868404,0.218084,0.276344
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407,3418303.68,37773.0,894731.77,82569.639202,69404.909528,123780.22,-2523571.91,-282.047871,0.201556,0.261747
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682,3261197.94,60211.0,1275752.89,76734.849173,110634.689261,257032.07,-1985445.05,-155.629281,0.298544,0.391191
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109,3023206.01,33521.0,1013228.79,102078.279297,61592.039796,257032.07,-2009977.22,-198.373481,0.242714,0.335150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750.0,2,2.64,0.0,0.00,0.000000,0.000000,27100.41,-2.64,-inf,0.000000,0.000000
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50.0,6,2.34,0.0,0.00,0.000000,0.000000,50293.62,-2.34,-inf,0.000000,0.000000
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50.0,2,1.48,2.0,1.98,0.990000,0.100000,14069.87,0.50,25.252525,1.000000,1.337838
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200.0,1,1.47,15.0,29.85,21.890000,3.150000,257032.07,28.38,95.075377,15.000000,20.306122


In [105]:
vendor_sales_summary.to_csv('vendor_sales_summary_Cleaned.csv', index=False)